Dataset Cleaning 

The main objectives for this notebooks are:
- Get familiar with the dataset
- Discover the data qualit y issues
- Fix the data quality issues 

In [1]:
import sys, os
import plotly.io as pio 

# Path needs to be added manually to read from another folder
path2add = os.path.normpath(os.path.abspath(os.path.join(os.path.dirname('__file__'), os.path.pardir, 'utils')))
if (not (path2add in sys.path)) :
    sys.path.append(path2add)


In [3]:
import polars as pl 
import plotly.express as px
from cleaning import (describe_objects, count_missing)


Data Ingestion

In [5]:
data = pl.read_csv("../data/Lead Scoring.csv")
print(data.shape)
data.head()

(9240, 37)


Prospect ID,Lead Number,Lead Origin,Lead Source,Do Not Email,Do Not Call,Converted,TotalVisits,Total Time Spent on Website,Page Views Per Visit,Last Activity,Country,Specialization,How did you hear about X Education,What is your current occupation,What matters most to you in choosing a course,Search,Magazine,Newspaper Article,X Education Forums,Newspaper,Digital Advertisement,Through Recommendations,Receive More Updates About Our Courses,Tags,Lead Quality,Update me on Supply Chain Content,Get updates on DM Content,Lead Profile,City,Asymmetrique Activity Index,Asymmetrique Profile Index,Asymmetrique Activity Score,Asymmetrique Profile Score,I agree to pay the amount through cheque,A free copy of Mastering The Interview,Last Notable Activity
str,i64,str,str,str,str,i64,i64,i64,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,i64,str,str,str
"""7927b2df-8bba-4d29-b9a2-b6e0be…",660737,"""API""","""Olark Chat""","""No""","""No""",0,0,0,0.0,"""Page Visited on Website""",null,"""Select""","""Select""","""Unemployed""","""Better Career Prospects""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""Interested in other courses""","""Low in Relevance""","""No""","""No""","""Select""","""Select""","""02.Medium""","""02.Medium""",15,15,"""No""","""No""","""Modified"""
"""2a272436-5132-4136-86fa-dcc88c…",660728,"""API""","""Organic Search""","""No""","""No""",0,5,674,2.5,"""Email Opened""","""India""","""Select""","""Select""","""Unemployed""","""Better Career Prospects""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""Ringing""",null,"""No""","""No""","""Select""","""Select""","""02.Medium""","""02.Medium""",15,15,"""No""","""No""","""Email Opened"""
"""8cc8c611-a219-4f35-ad23-fdfd26…",660727,"""Landing Page Submission""","""Direct Traffic""","""No""","""No""",1,2,1532,2.0,"""Email Opened""","""India""","""Business Administration""","""Select""","""Student""","""Better Career Prospects""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""Will revert after reading the …","""Might be""","""No""","""No""","""Potential Lead""","""Mumbai""","""02.Medium""","""01.High""",14,20,"""No""","""Yes""","""Email Opened"""
"""0cc2df48-7cf4-4e39-9de9-19797f…",660719,"""Landing Page Submission""","""Direct Traffic""","""No""","""No""",0,1,305,1.0,"""Unreachable""","""India""","""Media and Advertising""","""Word Of Mouth""","""Unemployed""","""Better Career Prospects""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""Ringing""","""Not Sure""","""No""","""No""","""Select""","""Mumbai""","""02.Medium""","""01.High""",13,17,"""No""","""No""","""Modified"""
"""3256f628-e534-4826-9d63-4a8b88…",660681,"""Landing Page Submission""","""Google""","""No""","""No""",1,2,1428,1.0,"""Converted to Lead""","""India""","""Select""","""Other""","""Unemployed""","""Better Career Prospects""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""Will revert after reading the …","""Might be""","""No""","""No""","""Select""","""Mumbai""","""02.Medium""","""01.High""",15,18,"""No""","""No""","""Modified"""


Data Profiling

In [6]:
print("Original shape:" ,  data.shape)
print("Columns:" , data.columns)

Original shape: (9240, 37)
Columns: ['Prospect ID', 'Lead Number', 'Lead Origin', 'Lead Source', 'Do Not Email', 'Do Not Call', 'Converted', 'TotalVisits', 'Total Time Spent on Website', 'Page Views Per Visit', 'Last Activity', 'Country', 'Specialization', 'How did you hear about X Education', 'What is your current occupation', 'What matters most to you in choosing a course', 'Search', 'Magazine', 'Newspaper Article', 'X Education Forums', 'Newspaper', 'Digital Advertisement', 'Through Recommendations', 'Receive More Updates About Our Courses', 'Tags', 'Lead Quality', 'Update me on Supply Chain Content', 'Get updates on DM Content', 'Lead Profile', 'City', 'Asymmetrique Activity Index', 'Asymmetrique Profile Index', 'Asymmetrique Activity Score', 'Asymmetrique Profile Score', 'I agree to pay the amount through cheque', 'A free copy of Mastering The Interview', 'Last Notable Activity']


In [7]:
# Identify categorical (object-like) columns
categorical_cols = [col for col in data.columns if data[col].dtype in [pl.Utf8, pl.Categorical]]
# Identify numerical columns
numerical_cols = [col for col in data.columns if data[col].dtype in [pl.Int32, pl.Int64, pl.UInt32, pl.UInt64, pl.Float32, pl.Float64]]

In [45]:
data.select(numerical_cols).describe()

statistic,Lead Number,Converted,TotalVisits,Total Time Spent on Website,Page Views Per Visit,Asymmetrique Activity Score,Asymmetrique Profile Score
str,f64,f64,f64,f64,f64,f64,f64
"""count""",9240.0,9240.0,9103.0,9240.0,9103.0,5022.0,5022.0
"""null_count""",0.0,0.0,137.0,0.0,137.0,4218.0,4218.0
"""mean""",617188.435606,0.38539,3.445238,487.698268,2.36282,14.306252,16.344883
"""std""",23405.995698,0.486714,4.854853,548.021466,2.161418,1.386694,1.811395
"""min""",579533.0,0.0,0.0,0.0,0.0,7.0,11.0
"""25%""",596485.0,0.0,1.0,12.0,1.0,14.0,15.0
"""50%""",615481.0,0.0,3.0,248.0,2.0,14.0,16.0
"""75%""",637387.0,1.0,5.0,936.0,3.0,15.0,18.0
"""max""",660737.0,1.0,251.0,2272.0,55.0,18.0,20.0


In [8]:
describe_objects(data)

Statistic,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11,column_12,column_13,column_14,column_15,column_16,column_17,column_18,column_19,column_20,column_21,column_22,column_23,column_24,column_25,column_26,column_27,column_28,column_29
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""Statistic""","""Prospect ID""","""Lead Origin""","""Lead Source""","""Do Not Email""","""Do Not Call""","""Last Activity""","""Country""","""Specialization""","""How did you hear about X Educa…","""What is your current occupatio…","""What matters most to you in ch…","""Search""","""Magazine""","""Newspaper Article""","""X Education Forums""","""Newspaper""","""Digital Advertisement""","""Through Recommendations""","""Receive More Updates About Our…","""Tags""","""Lead Quality""","""Update me on Supply Chain Cont…","""Get updates on DM Content""","""Lead Profile""","""City""","""Asymmetrique Activity Index""","""Asymmetrique Profile Index""","""I agree to pay the amount thro…","""A free copy of Mastering The I…","""Last Notable Activity"""
"""count""","""9240""","""9240""","""9204""","""9240""","""9240""","""9137""","""6779""","""7802""","""7033""","""6550""","""6531""","""9240""","""9240""","""9240""","""9240""","""9240""","""9240""","""9240""","""9240""","""5887""","""4473""","""9240""","""9240""","""6531""","""7820""","""5022""","""5022""","""9240""","""9240""","""9240"""
"""unique""","""9240""","""5""","""22""","""2""","""2""","""18""","""39""","""20""","""11""","""7""","""4""","""2""","""1""","""2""","""2""","""2""","""2""","""2""","""1""","""27""","""6""","""1""","""1""","""7""","""8""","""4""","""4""","""1""","""2""","""16"""
"""null_count""","""0""","""0""","""36""","""0""","""0""","""103""","""2461""","""1438""","""2207""","""2690""","""2709""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""3353""","""4767""","""0""","""0""","""2709""","""1420""","""4218""","""4218""","""0""","""0""","""0"""
"""top""","""1462884a-d722-492a-8457-98f453…","""Landing Page Submission""","""Google""","""No""","""No""","""Email Opened""","""India""","""Select""","""Select""","""Unemployed""","""Better Career Prospects""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""",null,null,"""No""","""No""","""Select""","""Mumbai""",null,null,"""No""","""No""","""Modified"""
"""freq""","""1""","""4886""","""2868""","""8506""","""9238""","""3437""","""6492""","""1942""","""5043""","""5600""","""6528""","""9226""","""9240""","""9238""","""9239""","""9239""","""9236""","""9233""","""9240""",null,null,"""9240""","""9240""","""4146""","""3222""",null,null,"""9240""","""6352""","""3407"""


In [ ]:
data.filter(pl.col("inter_api_access_duration(sec)").is_null())

Observations
- Some


In [ ]:
Impact
- Columns '' '' ''  '' need to be dropped as almost half of the values are missing